In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as scs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer

from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

from sklearn.metrics import r2_score
import myFunctions

In [2]:
df = pd.read_csv('~/flatiron/mod2/dsc-mod-2-project-v2-1-onl01-dtsc-ft-041320/csv/data_correlation_gone.csv',index_col = 0)

In [3]:

check_col = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot','distance_from_a_city']
all_indices = []
for col in check_col:
    indices = remove_outliers(df, col, threshold = 3)
    all_indices.extend(indices)
all_indices = np.unique(all_indices)

NameError: name 'remove_outliers' is not defined

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df.drop(all_indices,axis=0,inplace= True)

In [ ]:
df.corr()['price']

In [ ]:
sns.violinplot(x='price', y='condition', data=df, orient='h')
plt.show()

In [ ]:
sns.violinplot(x='price', y='grade', data=df, orient='h')
plt.show()

In [ ]:
features =  ['sqft_living','distance_from_a_city','sqft_lot', 'bedrooms','bathrooms']

In [ ]:
df[features].hist();

In [ ]:
tobox =['sqft_living','distance_from_a_city','sqft_lot']

In [ ]:
for feat in tobox:
    df[feat],l = scs.boxcox(df[feat])


In [ ]:
df[features].hist();

In [ ]:
for feat in features:
    df[feat] = scs.zscore(df[feat])


In [ ]:
df.columns

# Create booleans for Grade 4 - 11

In [ ]:
df['grade4'] = df.grade.apply(lambda x: 1 if x ==4 else 0)
df['grade5'] = df.grade.apply(lambda x: 1 if x ==5 else 0)
df['grade6'] = df.grade.apply(lambda x: 1 if x ==6 else 0)
df['grade7'] = df.grade.apply(lambda x: 1 if x ==7 else 0)
df['grade8'] = df.grade.apply(lambda x: 1 if x ==8 else 0)
df['grade9'] = df.grade.apply(lambda x: 1 if x ==9 else 0)
df['grade10'] = df.grade.apply(lambda x: 1 if x ==10 else 0)
df['grade11'] = df.grade.apply(lambda x: 1 if x ==11 else 0)


# Choosing Columns to Model

In [ ]:
x_cols = ['bedrooms','bathrooms','sqft_living','sqft_lot','distance_from_a_city',
           'grade4', 'grade5', 'grade6', 'grade7', 'grade8', 'grade9',
       'grade10', 'grade11']


# Building Model

In [ ]:
X = df[x_cols]
y = df['price']
model = sm.OLS(y,X).fit()
model.summary()

# Checking if p-values are less than 0.05 and getting rid of them

In [ ]:
check_p_values(model)

# Running a Second Iteration of the model

In [ ]:
X = df[x_cols]
y = df['price']
model = sm.OLS(y,X).fit()
model.summary()

# Checking for Multicollinearity using VIF

In [ ]:
VIFScore(x_cols)

# Getting rid of scores less than 5 

In [ ]:
vif_scores = VIFScore(x_cols)
x_cols = [x for x,vif in vif_scores if vif < 5]
print(len(vif_scores), len(x_cols))

In [ ]:
X = df[x_cols]
y = df['price']
model = sm.OLS(y,X).fit()
model.summary()

# Checking for Normality

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=scs.norm, line='45', fit=True)


# Checking for Homoscedasticity

In [ ]:
plt.scatter(model.predict(X), model.resid)
plt.plot(model.predict(X), [0 for i in range(len(df))])

# Removing price points outside the 95th percentile - Cutoff is 1million 

In [ ]:
df.price.hist()

In [ ]:
for i in range(80,100):
    q = i/100
    print("{} percentile: {}".format(q, df.price.quantile(q=q)))

In [ ]:
orig_tot = len(df)
df = df[df.price < 1000000.0] # Subsetting to remove extreme outliers
print('Percent removed:', (orig_tot -len(df))/orig_tot)


# Running our model again with new price points

In [ ]:
X = df[x_cols]
y = df['price']
model = sm.OLS(y,X).fit()
model.summary()

# Checking p-values again

In [ ]:
check_p_values(model)

# Another model

In [ ]:
build_ols(x_cols,'price',df)

# Checking again for multicollinearity

In [ ]:
# Your code here - Code a way to identify multicollinearity
VIFScore(x_cols)

In [ ]:
vif_scores = VIFScore(x_cols)
x_cols = [x for x,vif in vif_scores if vif < 5]
print(len(vif_scores), len(x_cols))

# Running our model one more time

In [ ]:
X = df[x_cols]
y = df['price']
model = sm.OLS(y,X).fit()
model.summary()

# Normality Assumption

**Residuals are more normal having cut the price at 1m**

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=scs.norm, line='45', fit=True)


In [ ]:
plt.scatter(model.predict(X), model.resid)
plt.plot(model.predict(X), [0 for i in range(len(df))])

In [ ]:
residuals = model.resid

Plotting Residauls to see distirbution

In [ ]:
plot_residuals(residuals)

# Cross Validation 

In [ ]:
ols = LinearRegression(fit_intercept=False, normalize=False, n_jobs=-1)

# Train test split

In [ ]:
make_ols_sklearn(X, y)

# Shuffling data and performing cross validation to check if R2 stays the same 

In [ ]:
X_shuff, y_shuff = shuffle(X, y)

In [ ]:
X_shuff, y_shuff = shuffle(X, y)
cv_scores = cross_val_score(ols, X_shuff, y_shuff, scoring='r2', cv=5, n_jobs=-1) 
cv_scores.mean()

In [ ]:
X_shuff, y_shuff = shuffle(X, y)
cv_scores = cross_val_score(ols, X_shuff, y_shuff, scoring='r2', cv=5, n_jobs=-1) 
cv_scores.mean()

In [ ]:
X_shuff, y_shuff = shuffle(X, y)
make_ols_sklearn(X_shuff, y_shuff)
cv_scores = cross_val_score(ols, X_shuff, y_shuff, scoring='r2', cv=5, n_jobs=-1) 
cv_scores.mean()

# Final Model

In [ ]:
model.summary()

In [ ]:
model.params

# plotting residuals against price

In [ ]:
residuals = model.resid
resid = pd.DataFrame(df['price'])
resid['residuals']  = residuals

In [ ]:
plt.figure(figsize= (10,10))

sns.scatterplot(y='residuals',x='price',data=resid)

Can see from the above graph that after price goes above approx 600,000 the residuals start to skew

# Checking how our model predicts our data

In [ ]:
ols = LinearRegression(fit_intercept=False,normalize=False)

In [ ]:
ols.fit(X,y)

In [ ]:
df['price'].mean()

In [ ]:
sample = df.loc[(df['price'] > 400000.00) & (df['price']<500000.00)]

In [ ]:
sample [:len(sample)];

In [ ]:
sample.shape

In [ ]:
sample = sample[x_cols];

In [ ]:
def predict_price(x_sample):
    cols =['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'distance_from_a_city',
 'grade4',
 'grade5',
 'grade6',
 'grade7',
 'grade8',
 'grade9',
 'grade10',
 'grade11']
    x_sample_final = x_sample[cols]
    return ols.predict(x_sample_final)[0]

In [ ]:
predict_list = []
for i in range(len(sample)):
    predict = ols.predict(sample[i:i+1])
    predict_list.extend(predict)

In [ ]:
df['price'].mean()

In [ ]:
np.mean(predict_list)

**For House prices in a sample of 3100 houses between USD 400,000 and USD 500,000. The model predicted USD 2000 away from the average price** 

In [ ]:
sample2 = df.loc[df['price']<800000.00]

In [ ]:
np.mean(sample2['price'])

In [ ]:
len(sample2)

In [ ]:
sample2 = sample2[x_cols]

In [ ]:
predict_list2 = []
for i in range(len(sample2)):
    predict = ols.predict(sample2[i:i+1])
    predict_list2.extend(predict)

In [ ]:
np.mean(predict_list2)

**For a Sample of 16078 houses, the average prediction was approx 11K off the average house price for sample**